In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 0.1.50


In [2]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Found the config file in: /home/jayaubuntudsvm/Desktop/AKSDeploymentTutorial/Keras_Tensorflow/aml_config/config.json
jayavienna
jayavienna
eastus2
edf507a2-6235-46c5-b560-fd463ba2e771


In [3]:
# choose a name for the run history container in the workspace
#run_history_name = 'train-on-local'

# project folder
#project_folder = './sample_projects/train-on-local'

In [4]:
from azureml.core import Experiment
experiment_name = 'train-on-local'
exp = Experiment(workspace=ws, name=experiment_name)

In [5]:
#import os
#from azureml.core import Project

#project = Project.attach(workspace_object = ws,
#                         history_name = run_history_name,
#                         directory = project_folder)

#print(project.project_directory, project.history.name, sep = '\n')

In [6]:
import os
script_folder = './samples/train-on-local'
os.makedirs(script_folder, exist_ok=True)

In [7]:
%%writefile $script_folder/train.py

#Develop the model
import tensorflow as tf
import keras
from resnet152 import ResNet152
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
import numpy as np
from PIL import Image
import wget

model = ResNet152(weights='imagenet')

#model.summary( )

wget.download('https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Lynx_lynx_poing.jpg/220px-Lynx_lynx_poing.jpg')

img_path = '220px-Lynx_lynx_poing.jpg'
#print(Image.open(img_path).size)
Image.open(img_path)

img = image.load_img(img_path, target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)

preds = model.predict(img)
print('Predicted:', decode_predictions(preds, top=3))

Overwriting ./samples/train-on-local/train.py


In [8]:
#User-managed runs

#from azureml.core.runconfig import RunConfiguration

# Editing a run configuration property on-fly.
#run_config = RunConfiguration.load(project_object = project, run_config_name = "local")

#run_config.environment.python.user_managed_dependencies = True
#run_config.prepare_environment = True

# You can choose a specific Python environment by pointing to a Python path 
#run_config.environment.python.interpreter_path = '/home/ninghai/miniconda3/envs/sdk2/bin/python'

In [9]:
#User-managed runs

from azureml.core.runconfig import RunConfiguration

# Editing a run configuration property on-fly.
run_config_user_managed = RunConfiguration()

run_config_user_managed.environment.python.user_managed_dependencies = True

# You can choose a specific Python environment by pointing to a Python path 
#run_config.environment.python.interpreter_path = '/home/ninghai/miniconda3/envs/sdk2/bin/python'

In [10]:
#%%time 
#from azureml.core.run import Run

#run = Run.submit(project_object = project,
#                 run_config = run_config,
#                 script_to_run = 'train.py')

# Shows output of the run on stdout.
#run.wait_for_completion(show_output = True)

In [11]:
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=script_folder, script='train.py', run_config=run_config_user_managed)
run = exp.submit(src)
run.wait_for_completion(show_output=True)

RunId: train-on-local_1537379692_e05a2b0e

Streaming azureml-logs/60_control_log.txt

Uploading experiment status to history service.
Adding run profile attachment azureml-logs/60_control_log.txt
Running: ['python', 'azureml-setup/run_script.py', 'python', 'azureml-setup/context_manager_injector.py', '-i', 'ProjectPythonPath:context_managers.ProjectPythonPath', '-i', 'OutputCollection:context_managers.RunHistory', 'train.py']
Logging experiment running status in history service.
Uploading experiment status to history service.
Adding run profile attachment azureml-logs/80_driver_log.txt

Streaming azureml-logs/80_driver_log.txt

Using TensorFlow backend.
2018-09-19 17:54:54.855149: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Predicted: [[('n02127052', 'lynx', 0.99590886), ('n02128385', 'leopard', 0.001150445), ('n02123159', 'tiger_cat', 0.0009417962)]]


The experiment completed success

{'runId': 'train-on-local_1537379692_e05a2b0e',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2018-09-19T17:54:52.635338Z',
 'properties': {'azureml.runsource': 'experiment',
  'ContentSnapshotId': '45acfa63-1906-4510-9579-cf6f11425b33'},
 'runDefinition': {'Script': 'train.py',
  'Arguments': [],
  'Framework': 0,
  'Target': 'local',
  'DataReferences': {},
  'JobName': None,
  'AutoPrepareEnvironment': True,
  'MaxRunDurationSeconds': None,
  'Environment': {'Python': {'InterpreterPath': 'python',
    'UserManagedDependencies': True,
    'CondaDependencies': {'name': 'project_environment',
     'dependencies': ['python=3.6.2',
      {'pip': ['--index-url https://azuremlsdktestpypi.azureedge.net/sdk-release/Preview/E7501C02541B433786111FE8E140CAA1',
        '--extra-index-url https://pypi.python.org/simple',
        'azureml-defaults']}]},
    'CondaDependenciesFile': None},
   'EnvironmentVariables': {'EXAMPLE_ENV_VAR': 'EXAMPLE_VALUE'},
   'Docker': {'BaseImage': '

In [12]:
# make sure you have helpers.py in the current folder
#import helpers
#print(helpers.get_run_history_url(run))

In [13]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
train-on-local,train-on-local_1537379692_e05a2b0e,azureml.scriptrun,Finalizing,Link to Azure Portal,Link to Documentation


In [14]:
# System managed runs
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

# Editing a run configuration property on-fly.
#run_config = RunConfiguration.load(project_object = project, run_config_name = "local")
run_config_system_managed = RunConfiguration()

# Use a new conda environment that is to be created from the conda_dependencies.yml file
#run_config.environment.python.user_managed_dependencies = False

# Automatically create the conda environment before the run
#run_config.prepare_environment = True

run_config_system_managed.environment.python.user_managed_dependencies = False
run_config_system_managed.prepare_environment = True

# add scikit-learn to the conda_dependencies.yml file
cd = CondaDependencies()
cd.add_conda_package('scikit-learn')
cd.add_conda_package('python=3.6')
cd.add_conda_package('tensorflow')
cd.add_conda_package('tornado==4.5.3')

cd.add_pip_package('papermill==0.14.1')
cd.add_pip_package('python-dotenv==0.9.0')
cd.add_pip_package('Pillow==5.2.0')
cd.add_pip_package('wget==3.2')
cd.add_pip_package('aiohttp==3.3.2')
cd.add_pip_package('toolz==0.9.0')
cd.add_pip_package('tqdm==4.23.4')
cd.add_pip_package('azure-cli==2.0.41')
#cd.add_pip_package('tensorflow')
cd.add_pip_package('keras==2.2.0')

#cd.save_to_file(project_dir = project_folder, conda_file_path = run_config.environment.python.conda_dependencies_file)

run_config_system_managed.environment.python.conda_dependencies = cd

In [15]:
#%%time 
#print("Submitting an experiment.")
#run = Run.submit(project_object = project,
#                 run_config = run_config, 
#                 script_to_run = "train.py")

# Shows output of the run on stdout.
#run.wait_for_completion(show_output = True)

In [16]:
src = ScriptRunConfig(source_directory=script_folder, script='train.py', run_config=run_config_system_managed)
run = exp.submit(src)
run.wait_for_completion(show_output = True)

RunId: train-on-local_1537379727_a6113561

Streaming azureml-logs/60_control_log.txt

Uploading experiment status to history service.
Adding run profile attachment azureml-logs/60_control_log.txt
Running: ['/home/jayaubuntudsvm/.azureml/envs/azureml_9e96feffd8b514609051cac0928c4c03/bin/python', 'azureml-setup/run_script.py', '/home/jayaubuntudsvm/.azureml/envs/azureml_9e96feffd8b514609051cac0928c4c03/bin/python', 'azureml-setup/context_manager_injector.py', '-i', 'ProjectPythonPath:context_managers.ProjectPythonPath', '-i', 'OutputCollection:context_managers.RunHistory', 'train.py']
Logging experiment running status in history service.
Uploading experiment status to history service.
Adding run profile attachment azureml-logs/80_driver_log.txt

Streaming azureml-logs/80_driver_log.txt

Using TensorFlow backend.
2018-09-19 17:55:30.054836: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2

{'runId': 'train-on-local_1537379727_a6113561',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2018-09-19T17:55:28.180337Z',
 'properties': {'azureml.runsource': 'experiment',
  'ContentSnapshotId': '45acfa63-1906-4510-9579-cf6f11425b33'},
 'runDefinition': {'Script': 'train.py',
  'Arguments': [],
  'Framework': 0,
  'Target': 'local',
  'DataReferences': {},
  'JobName': None,
  'AutoPrepareEnvironment': True,
  'MaxRunDurationSeconds': None,
  'Environment': {'Python': {'InterpreterPath': 'python',
    'UserManagedDependencies': False,
    'CondaDependencies': {'name': 'project_environment',
     'dependencies': ['python=3.6',
      {'pip': ['--index-url https://azuremlsdktestpypi.azureedge.net/sdk-release/Preview/E7501C02541B433786111FE8E140CAA1',
        '--extra-index-url https://pypi.python.org/simple',
        'azureml-defaults',
        'papermill==0.14.1',
        'python-dotenv==0.9.0',
        'Pillow==5.2.0',
        'wget==3.2',
        'aiohttp==3.3.2',
 

In [17]:
# import helpers.py
#import helpers
#print(helpers.get_run_history_url(run))

In [18]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
train-on-local,train-on-local_1537379727_a6113561,azureml.scriptrun,Finalizing,Link to Azure Portal,Link to Documentation


In [19]:
# Docker based execution

from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies

run_config_docker = RunConfiguration()

run_config_docker.environment.python.user_managed_dependencies = False
run_config_docker.prepare_environment = True
run_config_docker.environment.docker.enabled = True
run_config_docker.environment.docker.base_image = azureml.core.runconfig.DEFAULT_CPU_IMAGE

# Specify conda dependencies with scikit-learn
#cd = CondaDependencies.create(conda_packages=['scikit-learn'])

cd = CondaDependencies()
cd.add_conda_package('python=3.6')
cd.add_conda_package('tensorflow')
cd.add_conda_package('tornado==4.5.3')

cd.add_pip_package('papermill==0.14.1')
cd.add_pip_package('python-dotenv==0.9.0')
cd.add_pip_package('Pillow==5.2.0')
cd.add_pip_package('wget==3.2')
cd.add_pip_package('aiohttp==3.3.2')
cd.add_pip_package('toolz==0.9.0')
cd.add_pip_package('tqdm==4.23.4')
cd.add_pip_package('azure-cli==2.0.41')
cd.add_pip_package('keras==2.2.0')

run_config_docker.environment.python.conda_dependencies = cd

In [20]:
from azureml.core.runconfig import RunConfiguration
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=script_folder, script='train.py', run_config=run_config_docker)
run = exp.submit(src)

In [21]:
#Get run history details
run

Experiment,Id,Type,Status,Details Page,Docs Page
train-on-local,train-on-local_1537379769_b9d5180b,azureml.scriptrun,Running,Link to Azure Portal,Link to Documentation


In [22]:
run.wait_for_completion(show_output = True)

RunId: train-on-local_1537379769_b9d5180b

Streaming azureml-logs/60_control_log.txt

Uploading experiment status to history service.
Adding run profile attachment azureml-logs/60_control_log.txt
Logging experiment running status in history service.
Running: ['sudo', 'docker', 'run', '--name', 'train-on-local_1537379769_b9d5180b', '--rm', '-v', '/tmp/azureml_runs/train-on-local_1537379769_b9d5180b:/azureml-run', '-e', 'AZUREML_DOCKERFILE_PATH=azureml-setup/Dockerfile', '-e', 'AZUREML_CONDA_ENVIRONMENT_EXISTENCE_INDICATOR=$HOME/.azureml/locks/azureml_90ee2d497c42328141e4332a5eca2d6f', '-e', 'AZUREML_CONDA_ENVIRONMENT_PATH=/azureml-envs/azureml_90ee2d497c42328141e4332a5eca2d6f', '-e', 'AZUREML_CONDA_LOCK_PATH=$HOME/.azureml/locks/azureml_conda_lock', '-e', 'AZUREML_USER_CONDA_DIRECTORY=$HOME/.azureml', '-e', 'EXAMPLE_ENV_VAR=EXAMPLE_VALUE', '-e', 'AZUREML_CONTEXT_MANAGER_OUTPUTCOLLECTION=null', '-e', 'AZUREML_CONTEXT_MANAGER_PROJECTPYTHONPATH=null', '-e', 'AZUREML_RUN_TOKEN_EXPIRY=153798

{'runId': 'train-on-local_1537379769_b9d5180b',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2018-09-19T17:56:10.35489Z',
 'properties': {'azureml.runsource': 'experiment',
  'ContentSnapshotId': '45acfa63-1906-4510-9579-cf6f11425b33'},
 'runDefinition': {'Script': 'train.py',
  'Arguments': [],
  'Framework': 0,
  'Target': 'local',
  'DataReferences': {},
  'JobName': None,
  'AutoPrepareEnvironment': True,
  'MaxRunDurationSeconds': None,
  'Environment': {'Python': {'InterpreterPath': 'python',
    'UserManagedDependencies': False,
    'CondaDependencies': {'name': 'project_environment',
     'dependencies': ['python=3.6',
      {'pip': ['--index-url https://azuremlsdktestpypi.azureedge.net/sdk-release/Preview/E7501C02541B433786111FE8E140CAA1',
        '--extra-index-url https://pypi.python.org/simple',
        'azureml-defaults',
        'papermill==0.14.1',
        'python-dotenv==0.9.0',
        'Pillow==5.2.0',
        'wget==3.2',
        'aiohttp==3.3.2',
  